In [4]:
## do_shell_command
##%overwritefile
##%file:../src/do_shell_command.py
##%noruncode
    def do_shell_command(self,commands,cwd=None,shell=True,env=True,magics=None):
        # self._write_to_stdout(''.join((' '+ str(s) for s in commands)))
        try:
            if len(magics['_st']['replcmdmode'])>0:
                findObj= commands[0].split(" ",1)
                if findObj and len(findObj)>1:
                    cmd=findObj[0]
                    arguments=findObj[1]
                    cmdargs=[]
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', arguments):
                        cmdargs += [argument.strip('"')]
                    self._write_to_stdout(cmd)
                    self._write_to_stdout(''.join((' '+ str(s) for s in cmdargs))+"\n")
                    self._start_replprg(cmd,cmdargs,magics)
                    return
            p = RealTimeSubprocess(commands,
                                  self._write_to_stdout,
                                  self._write_to_stderr,
                                  self._read_from_stdin,cwd,shell,env=env)
            self.g_rtsps[str(p.pid)]=p
            if magics!=None and len(magics['showpid'])>0:
                self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
            while p.poll() is None:
                p.write_contents()
            # wait for threads to finish, so output is always shown
            p._stdout_thread.join()
            p._stderr_thread.join()
            del self.g_rtsps[str(p.pid)]
            p.write_contents()
            if p.returncode != 0:
                self._write_to_stderr("[MyPythonkernel] Error: Executable command exited with code {}\n".format(p.returncode))
            else:
                self._write_to_stdout("[MyPythonkernel] Info: command success.\n")
            return
        except Exception as e:
            self._write_to_stderr("[MyPythonkernel] Error:Executable command error! "+str(e)+"\n")


[MyPython] Info:file H:/Jupyter/Myjupyter-kernel/base/../src/do_shell_command.py created successfully
